In [1]:
class Option():
    
    def __init__(self):
        
        self.dataroot= r'/home/jara/DeepInPainting/paris_original_test' #image dataroot
        self.maskroot= r'/home/jara/DeepInPainting/mask/test_mask'
        self.batchSize= 1   # Need to be set to 1
        self.fineSize=256 # image size
        self.input_nc=3  # input channel size for first stage
        self.input_nc_g=6 # input channel size for second stage
        self.output_nc=3# output channel size
        self.ngf=64 # inner channel
        self.ndf=64# inner channel
        self.which_model_netD='basic' # patch discriminator
        self.which_model_netF='feature'# feature patch discriminator
        self.which_model_netG='unet_ipsr'# second stage network
        self.which_model_netP='unet_256'# first stage network
        self.triple_weight=1
        self.name='IPSR_inpainting'
        self.n_layers_D='3' # network depth
        self.gpu_ids=[0]
        self.model='csa_net'
        self.checkpoints_dir=r'/home/jara/trained_model/level0_epoch60' #
        self.norm='instance'
        self.fixed_mask=1
        self.use_dropout=False
        self.init_type='normal'
        self.mask_type='random'
        self.lambda_A=100
        self.threshold=5/16.0
        self.stride=1
        self.shift_sz=1 # size of feature patch
        self.mask_thred=1
        self.bottleneck=512
        self.gp_lambda=10.0
        self.ncritic=5
        self.constrain='MSE'
        self.strength=1
        self.init_gain=0.02
        self.cosis=1
        self.gan_type='lsgan'
        self.gan_weight=0.2
        self.overlap=4
        self.skip=0
        self.display_freq=1000
        self.print_freq=50
        self.save_latest_freq=5000
        self.save_epoch_freq=2
        self.continue_train=False
        self.epoch_count=1
        self.phase='train'
        self.which_epoch=60
        self.niter=20
        self.niter_decay=100
        self.beta1=0.5
        self.lr=0.0002
        self.lr_policy='lambda'
        self.lr_decay_iters=50
        self.isTrain=False

In [2]:
import time
from util.data_load import Data_load
from models.models import create_model
import torch
import os
import torchvision
from torch.utils import data
import torchvision.transforms as transforms
from IQA_pytorch import SSIM
import numpy as np
opt = Option()
transform_mask = transforms.Compose(
    [transforms.Resize((opt.fineSize,opt.fineSize)),
     transforms.ToTensor(),
    ])
transform = transforms.Compose(
    [transforms.Resize((opt.fineSize,opt.fineSize)),
     transforms.ToTensor(),
     transforms.Normalize(mean=[0.5] * 3, std=[0.5] * 3)])
transform_ref = transforms.Compose(
    [transforms.RandomResizedCrop((opt.fineSize,opt.fineSize),scale=(0.8,1.0),ratio=(1,1)),
     transforms.ColorJitter(brightness=0.1,contrast=0.1,saturation=0.1,hue=0.1),
     transforms.ToTensor(),
     transforms.Normalize(mean=[0.5] * 3, std=[0.5] * 3)])

dataset_test = Data_load(opt.dataroot, opt.maskroot,opt.dataroot, transform, transform_mask,transform_ref)
iterator_test = (data.DataLoader(dataset_test, batch_size=opt.batchSize,shuffle=True))
print(len(dataset_test))
model = create_model(opt)
total_steps = 0

2400
csa_net
initialize network with normal


/home/jara/DeepInPainting/models/networks.py:54: UserWarning: nn.init.normal is now deprecated in favor of nn.init.normal_.
  init.normal(m.weight.data, 0.0, gain)
/home/jara/DeepInPainting/models/networks.py:64: UserWarning: nn.init.constant is now deprecated in favor of nn.init.constant_.
  init.constant(m.bias.data, 0.0)


initialize network with normal
Loading pre-trained network!
model [CSAModel] was created


In [3]:
load_epoch=60
model.load(load_epoch)

In [4]:
save_dir = r'/home/jara/DeepInPainting/result'
if os.path.exists(save_dir) is False:
    os.makedirs(save_dir)

epoch=1
i=0
psnr_average, ssim_average = 0, 0
psnr_sum, ssim_sum = 0, 0
for image,mask,ref in (iterator_test):
    
    if epoch > 500:
        psnr_average = psnr_sum / 500
        ssim_average = ssim_sum / 500
        print("Finish")
        print('PSNR_average : %.2f, SSIM_average : %.3f' %(psnr_average, ssim_average))
        break

    iter_start_time = time.time()
    image=image.cuda()
    mask=mask.cuda()
    mask=mask[0][0]
    mask=torch.unsqueeze(mask,0)
    mask=torch.unsqueeze(mask,1)
    mask=mask.bool()

    model.set_input(image,mask,image) # it not only sets the input data with mask, but also sets the latent mask.
    model.set_ref_latent()
    model.set_gt_latent()
    model.test()
    
    real_A,ref_B,fake_B,fake_P,real_B=model.get_current_visuals()
    pic = (torch.cat([real_A,ref_B,fake_P,fake_B], dim=0) + 1) / 2.0
    
    torchvision.utils.save_image(pic, '%s/Epoch_(%d)_(%dof%d).jpg' % (
    save_dir, epoch, total_steps + 1, len(dataset_test)), nrow=2)
    
    psnr=0
    mse=0
    if type(real_B)==torch.Tensor:
        mse=torch.mean((real_B-fake_B)**2)
    else:
        mse=np.mean((real_B-fake_B)**2)
    
    if mse==0:
        psnr=100
    else:
        psnr=10*torch.log10((2**2)/mse)
    errors=model.get_error()
    
    mod=SSIM(channels=3)
    score=mod(real_B,fake_B,as_loss=False)
    print('%d. PSNR : %f, SSIM : %f' %(epoch,psnr,score))
    psnr_sum += psnr
    ssim_sum += score
    epoch=epoch+1

1. PSNR : 29.175392, SSIM : 0.862932
2. PSNR : 37.296474, SSIM : 0.989092
3. PSNR : 19.750727, SSIM : 0.525251
4. PSNR : 22.774019, SSIM : 0.866432
5. PSNR : 26.926327, SSIM : 0.755860
6. PSNR : 31.145649, SSIM : 0.791486
7. PSNR : 27.285223, SSIM : 0.832271
8. PSNR : 15.896793, SSIM : 0.497961
9. PSNR : 40.266647, SSIM : 0.957554
10. PSNR : 41.151474, SSIM : 0.969309
11. PSNR : 29.634350, SSIM : 0.948067
12. PSNR : 20.098789, SSIM : 0.610346
13. PSNR : 19.085949, SSIM : 0.696722
14. PSNR : 25.996605, SSIM : 0.849003
15. PSNR : 21.801699, SSIM : 0.734717
16. PSNR : 37.633137, SSIM : 0.964511
17. PSNR : 26.377260, SSIM : 0.776552
18. PSNR : 24.211334, SSIM : 0.755937
19. PSNR : 24.734762, SSIM : 0.802772
20. PSNR : 14.817868, SSIM : 0.461244
21. PSNR : 31.337442, SSIM : 0.911815
22. PSNR : 17.960203, SSIM : 0.530553
23. PSNR : 22.158634, SSIM : 0.601545
24. PSNR : 40.010525, SSIM : 0.971563
25. PSNR : 20.600950, SSIM : 0.548383
26. PSNR : 19.561598, SSIM : 0.620993
27. PSNR : 25.837706,

214. PSNR : 26.721958, SSIM : 0.784234
215. PSNR : 43.567280, SSIM : 0.990231
216. PSNR : 18.544596, SSIM : 0.556485
217. PSNR : 21.025818, SSIM : 0.614990
218. PSNR : 32.992695, SSIM : 0.875443
219. PSNR : 25.599682, SSIM : 0.845165
220. PSNR : 27.120167, SSIM : 0.908976
221. PSNR : 29.894794, SSIM : 0.888832
222. PSNR : 23.590260, SSIM : 0.676965
223. PSNR : 25.963469, SSIM : 0.850470
224. PSNR : 31.321737, SSIM : 0.918562
225. PSNR : 20.412945, SSIM : 0.645668
226. PSNR : 33.777691, SSIM : 0.963348
227. PSNR : 21.862593, SSIM : 0.609319
228. PSNR : 35.114082, SSIM : 0.969961
229. PSNR : 37.713703, SSIM : 0.964086
230. PSNR : 21.249939, SSIM : 0.571112
231. PSNR : 26.545832, SSIM : 0.851307
232. PSNR : 25.120459, SSIM : 0.800335
233. PSNR : 23.818180, SSIM : 0.794411
234. PSNR : 24.379166, SSIM : 0.716115
235. PSNR : 20.786055, SSIM : 0.678281
236. PSNR : 23.442602, SSIM : 0.834723
237. PSNR : 18.999884, SSIM : 0.608673
238. PSNR : 18.881500, SSIM : 0.610052
239. PSNR : 29.129009, SS

425. PSNR : 21.985691, SSIM : 0.723011
426. PSNR : 21.797010, SSIM : 0.640924
427. PSNR : 32.052616, SSIM : 0.916764
428. PSNR : 16.856873, SSIM : 0.527377
429. PSNR : 22.870964, SSIM : 0.760216
430. PSNR : 27.606813, SSIM : 0.885080
431. PSNR : 29.462004, SSIM : 0.789694
432. PSNR : 28.207376, SSIM : 0.880343
433. PSNR : 26.906609, SSIM : 0.812640
434. PSNR : 26.975403, SSIM : 0.878646
435. PSNR : 19.239483, SSIM : 0.583996
436. PSNR : 21.533709, SSIM : 0.572920
437. PSNR : 22.739067, SSIM : 0.734282
438. PSNR : 20.877630, SSIM : 0.623394
439. PSNR : 21.460758, SSIM : 0.719157
440. PSNR : 17.699942, SSIM : 0.626627
441. PSNR : 20.712582, SSIM : 0.665923
442. PSNR : 19.506630, SSIM : 0.613134
443. PSNR : 35.307384, SSIM : 0.949475
444. PSNR : 19.397417, SSIM : 0.708707
445. PSNR : 18.455980, SSIM : 0.633649
446. PSNR : 23.951408, SSIM : 0.647189
447. PSNR : 18.980593, SSIM : 0.503258
448. PSNR : 18.615669, SSIM : 0.603796
449. PSNR : 20.914879, SSIM : 0.747554
450. PSNR : 33.785290, SS